In [1]:
import pprint
import gcsfs
import numpy as np
import pandas as pd
import xarray as xr
from dateutil.relativedelta import relativedelta
import datetime
import cftime
import pysolar
from pysolar import *
import dask
import dask.delayed as delayed
from dask import compute
from matplotlib import pyplot as plt
import cartopy
import zarr

#from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = 12, 6
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'retina'")
import cartopy.crs as ccrs
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
fs = gcsfs.GCSFileSystem(token='anon', access='read_only')

#### Light calculations
Here we use pysolar python modules to perform the light calculations

In [7]:
grid_labels = ["gr"]

member_ids = [
    "r1i1p1f1",
    "r1i1p1f2",
    "r1i1p1f3",
    "r2i1p1f1",
    "r2i1p1f2",
    "r2i1p1f3",
    "r3i1p1f1",
    "r3i1p1f2",
    "r4i1p1f1",
    "r4i1p1f2",
    "r4i1p1f3",
    "r5i1p1f1",
    "r5i1p1f2",
    "r5i1p1f3",
    "r6i1p1f1",
    "r6i1p1f2",
    "r6i1p1f3",
    "r7i1p1f1",
    "r7i1p1f2",
    "r7i1p1f3",
    "r8i1p1f1",
    "r8i1p1f2",
    "r8i1p1f3",
    "r9i1p1f1",
    "r9i1p1f2",
    "r9i1p1f3",
    "r102i1p1f1",
    "r102i1p1f2",
    "r102i1p1f3",
]
member_ids = ["r1i1p1f1"]

experiment_ids = ["1pctCO2-bgc"]  #'abrupt-4xCO2',
institution_ids = ["NOAA-GFDL"]
source_ids = ["GFDL-ESM4"]
variable_ids = ["sithick", "siconc", "sisnthick", "sisnconc"]
table_ids = ["SImon", "SImon", "SImon", "SImon"]
# variable_ids = ["sithick"]

#### Query data
The dataset returned from the query need to be checked for time dimension. If the time array 
uses a relative date (0001 instead of 20001) then the values need to be added 2000 prior to converting to 
datetime64 format.

https://unidata.github.io/cftime/api.html
http://xarray.pydata.org/en/stable/time-series.html

In [8]:
def perform_query(query_string):
    df_sub = df.query(query_string)
    
    return df_sub

In [11]:
# Dicitionary to hold the queried variables
dset_dict = {}
first = True
for experiment_id in experiment_ids:
    for grid_label in grid_labels:
        for member_id in member_ids:

            # Historical
            query_string = "table_id=='Omon' and member_id=='{}' and grid_label=='{}' and experiment_id=='historical' and source_id=='GFDL-ESM4'".format(
                member_id, grid_label
            )
            #query_string = "source_id=='{}'and table_id=='{}' and member_id=='{}' and grid_label=='{}' and experiment_id=='esm-hist' and variable_id=='{}'".format(
            #    source_id, table_id, member_id, grid_label, variable_id
            #)
            print(
                "Running historical query on data: \n ==> {}\n".format(
                    query_string
                )
            )
            ds_hist = perform_query(query_string)
            print(ds_hist)
            # Future projection depending on choice in experiment_id
            query_string = "member_id=='{}' and grid_label=='{}' and experiment_id=='{}' and source_id=='GFDL-ESM4'".format(
                member_id,
                grid_label,
                experiment_id
            )
            print(
                "Running projections query on data: \n ==> {}\n".format(
                    query_string
                )
            )
            ds_proj = perform_query(query_string)
            print(ds_proj)
                   

Running historical query on data: 
 ==> table_id=='Omon' and member_id=='r1i1p1f1' and grid_label=='gr' and experiment_id=='historical' and source_id=='GFDL-ESM4'

      activity_id institution_id  source_id experiment_id member_id table_id  \
39123        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39125        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39126        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39127        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39129        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39131        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39132        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39136        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39138        CMIP      NOAA-GFDL  GFDL-ESM4    historical  r1i1p1f1     Omon   
39140        CMIP      NOAA-GFDL  GF